In [111]:
import os
import codecs
from docx import *
import docx2txt
from PIL import Image
from wand.image import Image as IMG
import pytesseract
import textract
import spacy
import pandas as pd
import itertools as it

nlp = spacy.load('en')
data = pd.read_excel("Skills.xlsx", header=0)
skill_list = list(data['Skill Names'])
skill_list = set(skill_list)
skill_list= [skill.lower() for skill in skill_list]
sorted(skill_list)

[' itunes connect',
 ' mobile design paradigm',
 '.net',
 'abstract factory',
 'accounting system',
 'accuracy and precision',
 'acquisition management',
 'activerecord',
 'acunetix',
 'acute coronary syndrome',
 'acute pain management',
 'ad campaigns',
 'adamsoft',
 'adaptability',
 'adapting to changing priorities',
 'adic',
 'administration and configuration',
 'adobe fireworks',
 'adobe framemaker',
 'adobe freehand',
 'adobe illustrator',
 'adobe indesign',
 'adobe photoshop',
 'adolescent health',
 'advanced features of carbonado',
 'adwords',
 'aes',
 'aggressive marketing',
 'agile',
 'agile development',
 'agile development process',
 'agile project methodology',
 'agilej structureviews',
 'agm',
 'ahnlab site guard',
 'aiml',
 'alasql',
 'aleph',
 'algorithm updates',
 'algorithmic',
 'algorithms',
 'all-source analysis',
 'alternative dispute resolution',
 'altium',
 'alv reporting',
 'amanda',
 'amateur photography',
 'amazon sqs',
 'amd am29000',
 'amira',
 'ampl',
 'amqp

In [112]:
def get_string(name):
    img=Image.open(name)
    utf8_text = pytesseract.image_to_string(img)
    utf8_text = str(utf8_text.encode('ascii', 'ignore'))
    return utf8_text

data_dir = os.path.join('Resumes')
resume_count = 0
resume_text = "all_text1.txt"
if 1==0:
    with codecs.open(resume_text,'w',encoding='utf_8') as resume_txt:
        for filename in os.listdir(data_dir):
            print(filename)

            s = ""
            if filename.endswith(".docx"):
                location = str(os.path.join(data_dir,filename))
                #document = Document(location)
                #for paragraph in document.paragraphs:
                #    s += str(paragraph.text.encode(errors='ignore'))
                s = docx2txt.process(location)
                resume_txt.write(s+'\\n'+'\n')
                resume_count += 1


            elif filename.endswith(".pdf"):
                utf8_text = ""

                temp_doc = textract.process(os.path.join(data_dir,filename))
                if len(temp_doc) < 50:
                    img=IMG(filename=os.path.join(data_dir,filename),resolution=200)

                    img.save(filename=os.path.join(data_dir,'temp','temp.jpg'))
                    for files in os.listdir(os.path.join(data_dir,'temp')):
                        utf8_text += get_string(os.path.join(data_dir,'temp',files))
                    for files in os.listdir(os.path.join(data_dir,'temp')):
                        os.remove(os.path.join(data_dir,'temp',files))
                    temp_doc = utf8_text

                if type(temp_doc)==str:
                    resume_txt.write(temp_doc+'\\n'+'\n')    
                else:
                    resume_txt.write(temp_doc.decode("utf-8")+'\\n'+'\n')    
                resume_count +=1
                
print(resume_count)

0


In [113]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

def punct_space(token):
    "helper that elimates puncations and whitespace"
    return token.is_punct or token.is_space

def line_review(filename):
    "read reviews from the file and un-escapes orignal line break"
    with codecs.open(filename,encoding='utf_8') as f:
        for res in f:
            yield res.replace('\\n','\n')

def lemmatized_sentence_corpus(filename):
    "use spacy to parse, lemmatize and yield sentences"
    for parsed_res in nlp.pipe(line_review(filename),batch_size=10000,n_threads=3):
        for sent in parsed_res.sents:
            yield u' '.join([token.lemma_ for token in sent if not punct_space(token)])
            
def get_related_terms(token,topn):
    arr =[]
    for word,similar in res2vec.wv.most_similar(positive = [token],topn=topn):
        print(word,':',round(similar,3))
        arr.append(word)
    return arr    

In [114]:
unigram_filepath = 'unigram_sentences.txt'
with codecs.open(unigram_filepath,'w',encoding='utf_8') as f:
    for sent in lemmatized_sentence_corpus(resume_text):
        f.write(sent+'\n')       
        

In [115]:
normal_res = LineSentence(unigram_filepath)
bigram_sentence = ''
normal_res2 = Phrases(normal_res)
with codecs.open('bigram.txt', 'w', encoding='utf_8') as f:
    for unigram_sentence in normal_res:        
        bigram_sentence =  ' '.join(normal_res2[unigram_sentence])
        f.write(bigram_sentence+'\n')
bigram_sentences = LineSentence('bigram.txt')

trigram_model = Phrases(bigram_sentences)
with codecs.open('trigram.txt', 'w', encoding='utf_8') as f:
    for unigram_sentence in bigram_sentences:        
        trigram_sentence =  ' '.join(trigram_model[unigram_sentence])
        f.write(trigram_sentence+'\n')
        
normal_res1 = LineSentence('trigram.txt')        

/home/jared/Parsing/parseVirt/lib/python3.5/site-packages/gensim/models/phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [116]:
from gensim.models import Word2Vec
word2vec_file = "vector_models"
if 1==0:
    res2vec = Word2Vec(normal_res,size=300,window=10,sg=1,workers=4,min_count=5)
    res2vec.save(word2vec_file)

    for i in range(1,15):
        res2vec.train(normal_res,total_examples=res2vec.corpus_count, epochs=res2vec.iter)
        res2vec.save(word2vec_file)
    
res2vec = Word2Vec.load(word2vec_file)
res2vec.init_sims()

In [117]:
ordered = [(term,voc.index,voc.count)for term, voc in res2vec.wv.vocab.items()]
ordered_terms,term_indices,term_count = zip(*ordered)
word_vectors = pd.DataFrame(res2vec.wv.syn0[term_indices,:],index=ordered_terms)

word_vectors.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
portion,0.272139,0.490600,-0.339325,0.212246,-0.295643,-0.100693,0.093254,-0.667629,-0.303324,0.061514,...,0.432418,-0.191583,0.480761,-0.028495,-0.030476,-0.140751,0.692747,-0.128718,-0.021175,-0.070898
brazil,0.323044,0.230862,-0.370784,-0.472313,-1.101111,-0.327409,-0.396169,-0.061429,-0.369953,-0.964593,...,0.057246,-0.058717,0.395460,-0.370003,-0.123298,-0.021105,0.392632,-0.514502,-0.066595,-0.851201
sequence,-0.161360,-0.324419,-0.012856,0.855552,-0.266395,-0.326030,-0.562629,-0.108521,-0.442791,-0.223648,...,0.408451,-0.177145,0.153521,-0.311002,0.053739,0.698397,-0.106383,0.063090,0.569674,0.176993
media,-0.104759,0.062560,-1.146530,0.234848,-0.603448,-0.498514,0.119498,0.684264,-0.071906,-0.166158,...,-0.423365,-0.250487,0.620142,-0.403606,-0.236812,0.165112,-0.424620,-0.294452,0.076237,-0.159287
verifone,-0.650822,-0.289868,-0.132865,0.338415,-0.603181,-0.486651,0.428210,-0.286760,-0.211088,-0.431017,...,-0.062531,-0.013798,0.601166,0.085414,-0.358076,0.411018,0.223352,-0.104798,0.409992,-0.529457


In [118]:
from sklearn.cluster import KMeans


word_vector = res2vec.wv.syn0
print(int(word_vector.shape[0]/50))
num_clusters = int(word_vector.shape[0]/50)

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vector )

111


In [119]:
word_centroid_map = dict(zip( res2vec.wv.index2word, idx ))
for cluster in range(0,num_clusters):
    #
    # Print the cluster number  
    print ("\nCluster:",cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for key,item in word_centroid_map.items():
        if( item == cluster ):
            words.append(key)
    print(words)


Cluster: 0
['identifying']

Cluster: 1
['patient', 'clinician', 'medicaid', 'doctor', 'medicare', 'deputy']

Cluster: 2
['tech', '24/7', 'noc', 'mlds', 'outage', 'teams', 'dashboards', 'managing', 'non', 'associated', 'sr', 'centers', 'liaise', 'manager/', 'sme', 'tickets', 'msde', 'direction', 'escalation', 'pmo', 'cutover', 'dr', 'pm', 'operations', 'projects', 'handled', 'collaboration', 'span', 'grc', 'sw', 's', 'coordinate', 'desk', 'shore', 'sales', 'interact', 'ordination', 'jaspersoft', 'ufms', 'readiness', 'analyst', 'odi', 'coe', 'si', 'managed', 'ordinate', 'ops', 'infrastructure', 'developers', 'openlink', 'crp', 'dpscs', 'call', 'supported', 'wramc', 'environments', 'laboratory', 'co', 'plm', 'qa', 'managers', 'act', 'areas', 'prod', 'making', 'pilot', 'wave', 'partner', 'oms', 'rebuild', 'tar', 'resolution', 'dba', 'forklift', 'verse', 'accountability', 'availability', 'rttt', 'rollout', 'primary', 'fab', 'msrb', 'hw', 'volume', 'stag', 'dev', 'lucent', 'trouble', '24x7'

['sdk']

Cluster: 15
['sample']

Cluster: 16
['stress', 'sit', '508', 'confidence', 'profiling', 'negative', 'conjoint', 'interval', 'interruption', 'acceptance', 'formal', 'harness', 'boundary', 'testing', 'biweekly', 'white', 'cleansing', 'mixture', 'ad', 'hypothesis', 'usability', 'news', 'facebook', 'ist', 'choice', 'gui', 'localization', 'sanity', 'regression', 'smoke', 'verification', '⦁', 'scenario', 'exploratory', 'hoc']

Cluster: 17
['prospective', 'optimize', 'tactical', '60%', 'feasible', 'acquire', 'maximize', 'pricing', 'improved', 'productivity', 'increase', 'segmentation', 'recommend', 'advantage', 'incentive', 'algae', 'lean', 'observe', 'turn', 'healthy', 'recur', 'stability', 'classmate', 'reduced', '15%', 'eye', 'recommendation', 'long', 'presence', 'brand', 'curtail', 'usage', 'rate', 'loop', '250', 'operational', 'reimbursement', 'demand', 'sustainable', 'competitor', 'knowledgeable', 'promise', 'kpi', 'interview', 'cost', 'suitable', 'thus', 'alignment', 'discount

['amdocs', 'navigator', 'cramer', 'oss', 'sync']

Cluster: 35
['block', 'attain', 'irs', 'tax', 'preparer', 'resident', 'license']

Cluster: 36
['mitigation']

Cluster: 37
['z', 'cobol', 'ii', 'mainframe', 'mf', 'erwin', 'jcl', 'sql*plus', 'cics']

Cluster: 38
['author', 'next', 'mis', 'survey', 'depth', 'flight', 'credential', 'dcs', 'demos', 'agile', 'custom', 'build', 'vast', 'approach', 'modernization', 'pathway', 'project', 'architect', 'effort', 'initiative', 'provide', 'aid', '\uf06e', 'r&d', 'exercise', 'the', 'e', 'integration', 'set', 'tier', '02/10/2012', 'additional', 'feature', 'customer', 'cross', '\ue00c', '/contribution', 'along', 'middle', 'study', 'centric', 'write', 'drive', 'database', 'enable', 'function', 'prototyping', 'multiple', 'level', 'i.e.', 'warehouse', 'ppm', 'varied', 'concept', 'lifecycle', 'contribute', 'method', 'external', 'current', 'isrweb', '’', 'instructional', 'range', '18', 'use', 'for', 'resource', 'control', 'azure', 'instructor', 'cube', 'se

['orleans', 'la', 'clerk', 'naval']

Cluster: 55
['b.e.', 'fluent', 'chemistry', 'ann', 'karnataka', 'psychology', 'diploma', 'principle', '4.0/4.0', '3.8', 'telecommunication', 'mathematics', 'masters', 'education', 'finance', 'statistics', 'b.s.', 'natural', 'behavioral', 'shonk', 'sagar', 'conversational', 'graduation', 'r.h.', 'leaders', '1992', 'double', 'interests', 'cum', 'ccna', 'detroit', 'cumulative', '3.7', 'endowed', '2018', 'graduate', 'physics', 'essential', 'colorado', 'arts', 'degree', 'mason', 'mandarin', 'russian', 'principles', 'ukraine', 'macroeconomics', 'cohort', 'electrical', 'courses', 'gmat', '3.4', 'bs', '0.00', 'anticipate', 'b.s', 'michigan', 'iit', '16.00', 'moscow', 'science', 'wayne', 'professionals', 'indiana', 'summa', 'gpa', 'robert', 'philosophy', 'polytechnic', 'graduated', 'literature', 'artificial', 'repeat', '12.00', 'relations', 'georgetown', '2019', 'master', 'shanghai', 'sciences', 'vellore', 'methods', 'b.', 'ugrd', 'expect', 'computer', 'cour

['sequence', 'lb', 'uft', 'progression', 'finding', 'off', 'rally', 'parametric', 'solid', 'proteus', 'retest', 'repair', 'keep', '\uf020', 'analysed', 'until', 'turnaround', 'cases', 'blackboard', 'investigate', 'servicenow', 'reproduce', 'thermo', 'severity', 'extreme', 'detect', 'continuously', 'extinguisher', 'compiler', 'circumstance', 'free', 'inform', 'tasks', 'kernel', 'openview', 'exam', 'depict', 'spirent', 'familiar', 'works', 'assign', 'rail', 'tbs', 'ticket', 'qtp', 'workaround', 'fire', 'priority', 'conclusion', 'driver', 'alm', 'diagnose', 'qc', 'drop', 'defect', 'pts', 'rc', 'waste', 'heat', 'triage', 'elementool', 'necessity', 'counter', '9.14', 'ride', 'robot', 'incident', 'atlassian', 'projection', 'testrail', 'stitching', 'loadrunner', 'tracker', 'drill', 'running', '12.0', 'pwm', 't', 'prompt', 'fix', 'bugzilla', 'hp', 'periodic', 'bug', 'partial', 'strength', 'color', 'mtm', 'track', 'itsm', 'status', 'scorecard', 'action', 'radar', 'close', 'ab', 'debug', 'bmc', 

['resources', 'human']

Cluster: 109
['hcm', 'vanilla', 'financials', 'peoplecode', '7.5', 'sandia', '8.9', 'elm', 'debugger', '8.4', '8.8', 'dms', 'mover', 'ps', '9.0', 'peopletools', '8.3', 'hrms', 'lm', 'engines', 'sqr', '8.51', 'peoplesoft', '9.1']

Cluster: 110
['video', 'game', 'touch', 'platform', 'tablet', 'com', 'xcode', 'music', 'android', 'anywhere', 'gear', 'device', 'simulator', 'iphone', 'platforms', 'emulator', 'ios', 'sms', 'ran', 'camera', 'accessibility', 'sencha', 'blackberry', 'galaxy', 'icloud', 'email', 'native', 'apps', 'skin', 'appium', 'apple', 'debugging', 'medium', 'smartphone', 'sharing', 'vr', 'prototyped', '∙test', 'mobile', 'desktop', 'app', 'adb', 'charles', 'samsung', 'ipad']


In [120]:
df_skill = pd.read_excel('Skills.xlsx',columns='Skills')
print(df_skill.head(),'\n')
skill_series = df_skill.T.squeeze().str.lower()
print(skill_series)

               Skill Names
0           iTunes Connect
1   Mobile Design Paradigm
2                     .NET
3         Abstract Factory
4        Accounting System 

0                         itunes connect
1                 mobile design paradigm
2                                   .net
3                       abstract factory
4                      accounting system
5                 accuracy and precision
6                 acquisition management
7                           activerecord
8                               acunetix
9                acute coronary syndrome
10                 acute pain management
11                          ad campaigns
12                              adamsoft
13                          adaptability
14       adapting to changing priorities
15                                  adic
16      administration and configuration
17                       adobe fireworks
18                      adobe framemaker
19                        adobe freehand
20              

In [121]:
d = {}
import operator
for cluster in range(0,num_clusters):
    #
    # Print the cluster number  
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for key,item in word_centroid_map.items():
        if( item == cluster ):
            words.append(key)
    cluster_series = pd.Series(words)  
    temp = pd.Series(list(set(cluster_series)&(set(skill_series))))
    d[cluster] = len(temp.values)
sorted_d = list(reversed(sorted(d.items(),key=operator.itemgetter(1))))
print(sorted_d)
print(sorted_d[1])
top_ten_skills = []
for i in range(0,3):
    key,val = sorted_d[i]
    top_ten_skills.append(key)
top_ten_skills = pd.Series(top_ten_skills)    
print(top_ten_skills)    

[(38, 25), (93, 19), (64, 19), (78, 15), (73, 13), (110, 7), (103, 7), (81, 7), (55, 6), (25, 6), (89, 5), (17, 5), (80, 4), (75, 4), (74, 4), (57, 4), (10, 4), (104, 3), (86, 3), (30, 3), (18, 3), (12, 3), (69, 2), (51, 2), (37, 2), (28, 2), (27, 2), (16, 2), (8, 2), (106, 1), (85, 1), (70, 1), (68, 1), (49, 1), (44, 1), (41, 1), (39, 1), (31, 1), (29, 1), (13, 1), (2, 1), (109, 0), (108, 0), (107, 0), (105, 0), (102, 0), (101, 0), (100, 0), (99, 0), (98, 0), (97, 0), (96, 0), (95, 0), (94, 0), (92, 0), (91, 0), (90, 0), (88, 0), (87, 0), (84, 0), (83, 0), (82, 0), (79, 0), (77, 0), (76, 0), (72, 0), (71, 0), (67, 0), (66, 0), (65, 0), (63, 0), (62, 0), (61, 0), (60, 0), (59, 0), (58, 0), (56, 0), (54, 0), (53, 0), (52, 0), (50, 0), (48, 0), (47, 0), (46, 0), (45, 0), (43, 0), (42, 0), (40, 0), (36, 0), (35, 0), (34, 0), (33, 0), (32, 0), (26, 0), (24, 0), (23, 0), (22, 0), (21, 0), (20, 0), (19, 0), (15, 0), (14, 0), (11, 0), (9, 0), (7, 0), (6, 0), (5, 0), (4, 0), (3, 0), (1, 0), (0

In [122]:
skills = []
other = []
for cluster in range(0,num_clusters):
    #
    # Print the cluster number  
    #
    # Find all of the words for that cluster number, and print them out
    if top_ten_skills.isin([cluster]).any():
        for key,item in word_centroid_map.items():
            if(item == cluster):
                skills.append(key)
    else:
        for key,item in word_centroid_map.items():
            if(item == cluster):
                other.append(key)    
skills                             

['author',
 'next',
 'mis',
 'survey',
 'depth',
 'flight',
 'credential',
 'dcs',
 'demos',
 'agile',
 'custom',
 'build',
 'vast',
 'approach',
 'modernization',
 'pathway',
 'project',
 'architect',
 'effort',
 'initiative',
 'provide',
 'aid',
 '\uf06e',
 'r&d',
 'exercise',
 'the',
 'e',
 'integration',
 'set',
 'tier',
 '02/10/2012',
 'additional',
 'feature',
 'customer',
 'cross',
 '\ue00c',
 '/contribution',
 'along',
 'middle',
 'study',
 'centric',
 'write',
 'drive',
 'database',
 'enable',
 'function',
 'prototyping',
 'multiple',
 'level',
 'i.e.',
 'warehouse',
 'ppm',
 'varied',
 'concept',
 'lifecycle',
 'contribute',
 'method',
 'external',
 'current',
 'isrweb',
 '’',
 'instructional',
 'range',
 '18',
 'use',
 'for',
 'resource',
 'control',
 'azure',
 'instructor',
 'cube',
 'session',
 'oversee',
 'time',
 'research',
 'unified',
 'be',
 'analyze',
 'many',
 'maturity',
 '\uf0b7',
 'more',
 'knowledge',
 'manage',
 'tiered',
 'corporate',
 'internet',
 'dynamic',


In [123]:
import nltk
from nltk.corpus import stopwords

special_characters = ['!','#', '$', '%','&','*','-', '/', '=','?',
                      '^','.','_','`', '{', '|', '}','~', "'", ',', '@', '(',')', ':', '•', '§' ]
# Processing text 

#tokenize sentences
resume_sents = nltk.sent_tokenize(resume_text)

#tokenize words
resume_words = [nltk.word_tokenize(sent) for sent in resume_sents]

In [124]:
resume_unigram=[]
for sentence in resume_words:
    sent = [w.lower() for w in sentence 
                    if w.lower() not in stopwords.words('english') and w.lower() not in special_characters]
    resume_unigram.append(sent)

In [125]:
#label skills in the resume

def labeled_word (sentence):
    labels=[]
    for word in sentence:
        if word in skill_list:
            labels.append((word, 'skill'))
        else:
            labels.append((word, 'not skill'))
    return labels


In [126]:
labeled_words=[labeled_word(sentence) for sentence in resume_unigram]

In [127]:
def similar_prob(word):
    count = 0
    terms = get_related_terms(word,25)
    for w in terms:
        if skill_series.isin([w]).any():
            print(w)
            count+=1
    return count/25        

In [128]:
similar_prob('python')

bash : 0.435
3-sectored : 0.428
verilog : 0.416
perl : 0.385
impala : 0.381
palisade : 0.38
precisiontree : 0.372
languag : 0.372
kornshell : 0.367
ocaml : 0.36
jigsaw : 0.358
stattools : 0.358
nodejs : 0.358
glimmix : 0.356
html5 : 0.356
solvertable : 0.355
myeclipse : 0.355
yarn : 0.354
minitab : 0.353
mandarin : 0.35
languages : 0.346
http://daveray.net : 0.344
mapreduce : 0.344
matlab : 0.343
client- : 0.342
perl
nodejs
matlab


0.12

In [129]:
def in_skill_cluster(word):
    if word in skills:
        return True
    return False

In [130]:
in_skill_cluster('java')

True

In [131]:
#extract featurres of skills 
def extract_features (sentence, i):
    features={}
    #first feature: evaluate if that word is in skill list
    features["({})in_skill_list".format(sentence[i])]= (sentence[i] in skill_list)
    features["probality_of_similar_words_skills"] = similar_prob(sentence[i])
    #if the word is in begining of the sentence, return <Start> for prev_word
    if i==0 and len(sentence)-1 != 0:
        features["prev_word_in_skill_list"]= '<Start>'
        features["next_word_in_skill_list"]= (sentence[i+1] in skill_list)
    
    #if the word is in begining of the sentence, return <End> for next_word
    elif i == len(sentence)-1 and  i != 0:
        features["prev_word_in_skill_list"]= (sentence[i-1] in skill_list)
        features["next_word_in_skill_list"]= '<End>'
    
    #if the sentence has only 1 word, return False for both prev_word and next_word
    elif i==0 and len(sentence)-1 == 0:
        features["prev_word_in_skill_list"]= False
        features["next_word_in_skill_list"]= False
    else:
        features["prev_word_in_skill_list"]= (sentence[i-1] in skill_list)
        features["next_word_in_skill_list"]= (sentence[i+1] in skill_list)
    return features

In [132]:
featuresets=[]
for labeled_sent in labeled_words:
    unlabeled_sent = [word[0] for word in labeled_sent]
    for i, (w, label) in enumerate(labeled_sent):
        featuresets.append((extract_features(unlabeled_sent, i), label))
                                                                   

KeyError: "word 'all_text1.txt' not in vocabulary"

In [ ]:
size = int(len(featuresets)*0.1)
train_set = featuresets[size:]
test_set = featuresets[:size]

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
nltk.classify.accuracy(classifier, test_set)

In [ ]:
classifier.show_most_informative_features(15)